In [405]:
import pandas as pd
import pulp as lp
import numpy as np
from operator import iadd
from functools import reduce
from typing import Sequence, Any, DefaultDict, List

In [406]:
def create_prob(prob_name: str, sense: int) -> lp.LpProblem:
    return lp.LpProblem(prob_name, sense)


def add_obj_fn(lp_prob: lp.LpProblem, dvar: lp.LpAffineExpression) -> lp.LpProblem:
    return iadd(lp_prob, dvar)


def add_constraint(lp_prob: lp.LpProblem, constrs: Sequence[lp.LpConstraint]) -> lp.LpProblem:
   return reduce(iadd, constrs, lp_prob)


def head(x: Sequence) -> Any:
    return x[0]

def to_str(indnum, activity) -> str:
    return f'{indnum} - {activity}'

In [407]:
df = pd.read_csv('data/dataset.csv')
source_df = pd.read_csv('data/dataset_source_cf.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)
source_df.drop(['Unnamed: 0', 'X__1'], axis=1, inplace=True)

In [408]:
df.head()

,Indnum,Group,Activity,Units,Consumption,Quality_of_Life_Importance__1_10,solar_powered_water_heater,gas_water_heater,electric_water_heater_peak_hour,electric_water_heater_off_peak,gas,natural_gas,hybrid,electric_peak_hours,electric_off_peak_hours,jetfuel,TCF,TrQL
0,1,1,Household heating => 70F,hours,2,88,0.0,0.0,0.0,0.0,0.0,0.000436,0,0.0,0.0,0.0,0.000436,176
1,1,1,Household heating < 70F,hours,10,85,0.0,0.0,0.0,0.0,0.0,0.000872,0,0.0,0.0,0.0,0.000872,850
2,1,1,Use of heat pump,hours,0,50,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0
3,1,1,Use of air conditioner,hours,20,45,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,900
4,1,2,shower - short,count,5,98,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,490


In [409]:
source_df

,Activity,Per,solar_powered_water_heater,gas_water_heater,electric_water_heater_peak_hour,electric_water_heater_off_peak,gas,natural_gas,jetfuel,waste management,hybrid,electric_peak_hours,electric_off_peak_hours
0,Household heating => 70F,hour,0.000000,0.000000,0.000000,0.000000,0.000000,0.000436,0.000000,0.0000,0.000000,0.000650,0.000542
1,Household heating < 70F,hour,0.000000,0.000000,0.000000,0.000000,0.000000,0.000872,0.000000,0.0000,0.000000,0.000923,0.000901
2,Use of heat pump,hour,0.000000,0.000000,0.000000,0.000000,0.000000,0.001074,0.000000,0.0000,0.000000,0.001229,0.001188
3,Use of air conditioner,hour,0.000000,0.000000,0.000000,0.000000,0.000000,0.000598,0.000000,0.0000,0.000000,0.007980,0.000721
4,shower - short,activity,0.000012,0.000102,0.000232,0.000199,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
5,shower - long (> 3 min),activity,0.000017,0.000149,0.000354,0.000312,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
6,bath,activity,0.000088,0.000254,0.000412,0.000368,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
7,wash-up,activity,0.000004,0.000042,0.000067,0.000055,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
8,use of dishwasher,activity,0.000025,0.000165,0.000398,0.000311,0.000000,0.000000,0.000000,0.0000,0.000000,0.000084,0.000078
9,use of clothes washer,activity,0.000033,0.000199,0.000433,0.000382,0.000000,0.000154,0.000000,0.0000,0.000000,0.000102,0.000093


In [410]:
indv_df = df.drop('Group', axis=1).loc[df['Indnum'] == 346]

In [411]:
indv_df.head()

,Indnum,Activity,Units,Consumption,Quality_of_Life_Importance__1_10,solar_powered_water_heater,gas_water_heater,electric_water_heater_peak_hour,electric_water_heater_off_peak,gas,natural_gas,hybrid,electric_peak_hours,electric_off_peak_hours,jetfuel,TCF,TrQL
9315,346,Household heating => 70F,count,8,91,0.0,0.000000,0.0,0.0,0.0,0.000436,0,0.0,0.0,0.0,0.000436,728
9316,346,Household heating < 70F,count,5,85,0.0,0.000000,0.0,0.0,0.0,0.000872,0,0.0,0.0,0.0,0.000872,425
9317,346,Use of heat pump,count,0,46,0.0,0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0
9318,346,Use of air conditioner,count,25,50,0.0,0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,1250
9319,346,shower - short,count,5,82,0.0,0.000102,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000102,410


In [412]:
gp_model = create_prob('Wells Fargo Challenge', lp.LpMinimize)

## Decision Variables

In [413]:
indv_nums = indv_df.Indnum.unique()
actv_names = indv_df.Activity.unique()
#actv_names = [actv_names[0]]
M = 100
sources = np.array([
  "solar_powered_water_heater",
  "gas_water_heater",
  "electric_water_heater_peak_hour",
  "electric_water_heater_off_peak",
  "gas",
  "natural_gas",
  "hybrid",
  "electric_peak_hours",
  "electric_off_peak_hours",
  "jetfuel",
  "waste management"
])

In [414]:
source_indexes = [
    (f'{indv} - {activity}', source)
    for indv in indv_nums
    for activity in actv_names
    for source in sources
]

S_ijk = lp.LpVariable.dicts('S_ijk', source_indexes, lowBound=0)

## Objective Function

### $Z_{min} = \sum S_{ijk} *  SCF_{ijk} * C_{ij}$

where 
* i is individual
* j is activity
* k is source
* _C_ is the consumption per unit of an activity
* _SCF_ is the carbon footprint per source

In [430]:
d_vars = []

for indv in indv_nums:
    for activity in actv_names:
        consumption: np.ndarray = indv_df.loc[indv_df['Activity'] == activity, 'Consumption'].values
            
        for source in sources:
            source_cf: np.ndarray = source_df.loc[source_df['Activity'] == activity, source].values
            print(head(consumption), '*', head(source_cf))
            source_cf = M if head(source_cf) == 0.0 else head(source_cf)
            d_vars.append(S_ijk[(to_str(indv, activity), source)] * float(source_cf) * head(consumption))
            print(activity, '|', source, '|', d_vars[len(d_vars) - 1])


obj_fn = lp.lpSum(sum(d_vars))

#obj_fn

8 * 0.0
Household heating => 70F | solar_powered_water_heater | 800.0*S_ijk_('346___Household_heating_=__70F',_'solar_powered_water_heater')
8 * 0.0
Household heating => 70F | gas_water_heater | 800.0*S_ijk_('346___Household_heating_=__70F',_'gas_water_heater')
8 * 0.0
Household heating => 70F | electric_water_heater_peak_hour | 800.0*S_ijk_('346___Household_heating_=__70F',_'electric_water_heater_peak_hour')
8 * 0.0
Household heating => 70F | electric_water_heater_off_peak | 800.0*S_ijk_('346___Household_heating_=__70F',_'electric_water_heater_off_peak')
8 * 0.0
Household heating => 70F | gas | 800.0*S_ijk_('346___Household_heating_=__70F',_'gas')
8 * 0.000436
Household heating => 70F | natural_gas | 0.003488*S_ijk_('346___Household_heating_=__70F',_'natural_gas')
8 * 0.0
Household heating => 70F | hybrid | 800.0*S_ijk_('346___Household_heating_=__70F',_'hybrid')
8 * 0.00065
Household heating => 70F | electric_peak_hours | 0.0052*S_ijk_('346___Household_heating_=__70F',_'electric_peak

use of clothes dryer | solar_powered_water_heater | 200.0*S_ijk_('346___use_of_clothes_dryer',_'solar_powered_water_heater')
2 * 0.0
use of clothes dryer | gas_water_heater | 200.0*S_ijk_('346___use_of_clothes_dryer',_'gas_water_heater')
2 * 0.0
use of clothes dryer | electric_water_heater_peak_hour | 200.0*S_ijk_('346___use_of_clothes_dryer',_'electric_water_heater_peak_hour')
2 * 0.0
use of clothes dryer | electric_water_heater_off_peak | 200.0*S_ijk_('346___use_of_clothes_dryer',_'electric_water_heater_off_peak')
2 * 0.0
use of clothes dryer | gas | 200.0*S_ijk_('346___use_of_clothes_dryer',_'gas')
2 * 0.00018700000000000002
use of clothes dryer | natural_gas | 0.00037400000000000004*S_ijk_('346___use_of_clothes_dryer',_'natural_gas')
2 * 0.0
use of clothes dryer | hybrid | 200.0*S_ijk_('346___use_of_clothes_dryer',_'hybrid')
2 * 0.000132
use of clothes dryer | electric_peak_hours | 0.000264*S_ijk_('346___use_of_clothes_dryer',_'electric_peak_hours')
2 * 0.000122
use of clothes drye

air travel - small  plane (<50 seats) | electric_off_peak_hours | 0
0 * 0.000408
air travel - small  plane (<50 seats) | jetfuel | 0
0 * 0.0
air travel - small  plane (<50 seats) | waste management | 0
344 * 0.0
car trips- self only | solar_powered_water_heater | 34400.0*S_ijk_('346___car_trips__self_only',_'solar_powered_water_heater')
344 * 0.0
car trips- self only | gas_water_heater | 34400.0*S_ijk_('346___car_trips__self_only',_'gas_water_heater')
344 * 0.0
car trips- self only | electric_water_heater_peak_hour | 34400.0*S_ijk_('346___car_trips__self_only',_'electric_water_heater_peak_hour')
344 * 0.0
car trips- self only | electric_water_heater_off_peak | 34400.0*S_ijk_('346___car_trips__self_only',_'electric_water_heater_off_peak')
344 * 0.000551
car trips- self only | gas | 0.189544*S_ijk_('346___car_trips__self_only',_'gas')
344 * 0.0
car trips- self only | natural_gas | 34400.0*S_ijk_('346___car_trips__self_only',_'natural_gas')
344 * 0.000332
car trips- self only | hybrid | 0

## Constraints

### $S_{ijk} == 1$

where 
* i is individual
* j is activity
* n = 1002
* i = 1...n
* j = 1...27
* k = 1...10

In [416]:
# indv_source_conds = []

# for indv in indv_nums:
#     for source in sources:
#         for activity in actv_names:
#             indv_source_conds.append(S_ijk[(to_str(indv, activity), source)] == 1)

# indv_source_conds

### $\sum_k^m S_{ijk} == 1$
where 
* i is individual
* j is activity
* n = 1002
* i = 1...n
* j = 1...27
* m = 10

In [417]:
sum_source_conds = []

for indv in indv_nums:
    for activity in actv_names:
        sum_source = []
        for source in sources:
            sum_source.append(S_ijk[(to_str(indv, activity), source)])
            
        sum_source_conds.append(lp.lpSum(sum_source) == 1)
        
sum_source_conds

[1*S_ijk_('346___Household_heating_=__70F',_'electric_off_peak_hours') + 1*S_ijk_('346___Household_heating_=__70F',_'electric_peak_hours') + 1*S_ijk_('346___Household_heating_=__70F',_'electric_water_heater_off_peak') + 1*S_ijk_('346___Household_heating_=__70F',_'electric_water_heater_peak_hour') + 1*S_ijk_('346___Household_heating_=__70F',_'gas') + 1*S_ijk_('346___Household_heating_=__70F',_'gas_water_heater') + 1*S_ijk_('346___Household_heating_=__70F',_'hybrid') + 1*S_ijk_('346___Household_heating_=__70F',_'jetfuel') + 1*S_ijk_('346___Household_heating_=__70F',_'natural_gas') + 1*S_ijk_('346___Household_heating_=__70F',_'solar_powered_water_heater') + 1*S_ijk_('346___Household_heating_=__70F',_'waste_management') + -1 = 0,
 1*S_ijk_('346___Household_heating_<_70F',_'electric_off_peak_hours') + 1*S_ijk_('346___Household_heating_<_70F',_'electric_peak_hours') + 1*S_ijk_('346___Household_heating_<_70F',_'electric_water_heater_off_peak') + 1*S_ijk_('346___Household_heating_<_70F',_'elec

In [418]:
gp_model = add_obj_fn(gp_model, obj_fn)
#gp_model = add_constraint(gp_model, indv_source_conds)
gp_model = add_constraint(gp_model, sum_source_conds)

In [419]:
gp_model.solve()

1

In [420]:
lp.LpStatus[gp_model.status]

'Optimal'

In [421]:
lp.value(gp_model.objective)

1.48585

In [422]:
for indv in indv_nums:
    for activity in actv_names:
        sum_source = []
        for source in sources:
            print(activity, f'| {source}: ', lp.value(S_ijk[(to_str(indv, activity), source)]))

Household heating => 70F | solar_powered_water_heater:  0.0
Household heating => 70F | gas_water_heater:  0.0
Household heating => 70F | electric_water_heater_peak_hour:  0.0
Household heating => 70F | electric_water_heater_off_peak:  0.0
Household heating => 70F | gas:  0.0
Household heating => 70F | natural_gas:  1.0
Household heating => 70F | hybrid:  0.0
Household heating => 70F | electric_peak_hours:  0.0
Household heating => 70F | electric_off_peak_hours:  0.0
Household heating => 70F | jetfuel:  0.0
Household heating => 70F | waste management:  0.0
Household heating < 70F | solar_powered_water_heater:  0.0
Household heating < 70F | gas_water_heater:  0.0
Household heating < 70F | electric_water_heater_peak_hour:  0.0
Household heating < 70F | electric_water_heater_off_peak:  0.0
Household heating < 70F | gas:  0.0
Household heating < 70F | natural_gas:  1.0
Household heating < 70F | hybrid:  0.0
Household heating < 70F | electric_peak_hours:  0.0
Household heating < 70F | electr